In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
!pip install transformers>=4.41.0

In [ ]:
import torch
print("GPU 사용 가능 여부:", torch.cuda.is_available())

GPU 사용 가능 여부: True


In [ ]:
!pip install torch transformers==4.40.0 accelerate openpyxl

import torch
import transformers
import json
import pandas as pd

model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)
pipeline.model.eval()

with open('./economy_ver3.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def generate_quiz(term, description):
  PROMPT = f"""

  용어: {term}
  설명: {description}

  각 용어와 설명을 바탕으로 O/X 문제를 만들어주세요.
  하나의 용어에 대해 상, 중, 하 난이도를 가지는 퀴즈를 총 3개씩 만들어 주세요.

  예를 들어서,

  term: 가계부실위험지수(HDRI)
  description: 가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상 환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다.

  이렇게 주어져 있다면,다음과 같은 퀴즈를 만들어주세요.

  난이도 상 문제: 가계부실위험지수(HDRI)는 가구의 DSR과 DTA가 각각 50%, 120%일 때 100의 값을 갖도록 설정되어 있다.
  난이도 상 정답: X
  난이도 중 문제: 가계부실위험지수(HDRI)는 가구의 소득 흐름과 자산만을 고려하여 가계부채의 부실위험을 평가하는 지표이다.
  난이도 중 정답: X
  난이도 하 문제: 가계부실위험지수란 가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실위험을 평가하는 지표이다.
  난이도 하 정답: O

  원하는 결과 형식은 다음과 같아요.
  term: <용어>
  description: <설명>
  quiz_high: <난이도 상 문제>
  answer_high: <난이도 상 정답>
  quiz_mid: <난이도 중 문제>
  answer_mid: <난이도 중 정답>
  quiz_low: <난이도 하 문제>
  answer_low: <난이도 하 정답>

  """

  outputs = pipeline(
      PROMPT,
      max_new_tokens=512,
      do_sample=True,
      temperature=0.7,
      top_p=0.85
  )

  return outputs[0]["generated_text"]

def extract_quiz(quiz_output, key):
    try:
        # 각 질문과 정답 부분을 키워드로 분리
        start_idx = quiz_output.index(f"{key}:") + len(f"{key}:")
        end_idx = quiz_output.index("\n", start_idx)
        return quiz_output[start_idx:end_idx].strip()
    except ValueError:
        return None

quiz_data = []

for item in data:
    term = item["term"]
    description = item["description"]

    print(f"🚀 퀴즈 생성 중: {term}")

    try:
        quiz_output = generate_quiz(term, description)
        print("생성된 퀴즈 출력:", quiz_output)

        quiz_data.append({
            "term": term,
            "description": description,
            "quiz_high": extract_quiz(quiz_output, "quiz_high"),
            "answer_high": extract_quiz(quiz_output, "answer_high"),
            "quiz_mid": extract_quiz(quiz_output, "quiz_mid"),
            "answer_mid": extract_quiz(quiz_output, "answer_mid"),
            "quiz_low": extract_quiz(quiz_output, "quiz_low"),
            "answer_low": extract_quiz(quiz_output, "answer_low"),
        })
    except Exception as e:
        print(f"🚨 오류 발생: {term} - {e}")
        continue

with open('quiz.json', 'w', encoding='utf-8') as f:
    json.dump(quiz_data, f, ensure_ascii=False, indent=4)

df = pd.DataFrame(quiz_data)
df.to_excel('quiz.xlsx', index=False)

print('✅ 퀴즈 생성 완료!')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


🚀 퀴즈 생성 중: 가계부실위험지수(HDRI)
생성된 퀴즈 출력: 

  용어: 가계부실위험지수(HDRI)
  설명: 가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상 환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다.

  각 용어와 설명을 바탕으로 O/X 문제를 만들어주세요.
  하나의 용어에 대해 상, 중, 하 난이도를 가지는 퀴즈를 총 3개씩 만들어 주세요.

  예를 들어서,

  term: 가계부실위험지수(HDRI)
  description: 가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상 환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 자산 측면에서 취약한 ‘고DTA가구’, 소득 측면